In [483]:
import pandas as pd

In [484]:
crime = pd.read_csv("./correlation_ready.csv",index_col=0)
crime.head()

,pointId,wall,lives,building,infrastructure,road,sidewalk,sky,green,transportation,publicservice,crime_counts
0,1,0.030779,0.003141,0.163661,0.020548,0.118350,0.033204,0.291897,0.201790,0.017978,0.000147,5
1,2,0.047397,0.008571,0.367944,0.009722,0.108147,0.035982,0.209341,0.089528,0.003407,0.000034,0
2,3,0.034286,0.003337,0.178823,0.008427,0.237291,0.061395,0.024113,0.378118,0.006196,0.000012,8
3,4,0.019361,0.001505,0.331703,0.004485,0.274849,0.073919,0.099125,0.107871,0.027312,0.000077,29
4,5,0.051927,0.002919,0.139440,0.039439,0.035637,0.003948,0.092139,0.446469,0.020049,0.000054,2


In [485]:
Vars=["wall","lives","building","infrastructure","road","sidewalk","sky","green","transportation","publicservice"]
selectCrime2 = crime
selectCrime2[Vars] = selectCrime2[Vars].apply(lambda x: round((x-x.min())/(x.max()-x.min()),2))

selectCrime2 = pd.melt(selectCrime2, id_vars=['pointId','crime_counts'])
selectCrime2 = selectCrime2.reset_index()
selectCrime2.head()

,index,pointId,crime_counts,variable,value
0,0,1,5,wall,0.19
1,1,2,0,wall,0.29
2,2,3,8,wall,0.21
3,3,4,29,wall,0.12
4,4,5,2,wall,0.32


df = selectCrime2.groupby(["variable","value"]).sum("count").reset_index()
df['crimeType'] = "All"
selectCrime2 = df.append(selectCrime2, ignore_index = True)

selectCrime2

selection3 = selectCrime2.loc[(selectCrime2["variable"]=="green")]

chart=alt.Chart(selection3).mark_point(
).encode(x='value:Q',y='count:Q')

chart + chart.transform_regression('value','count').mark_line()

# Regression

In [486]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm

np.random.seed(9876789)

In [487]:
selectCrimeCI.head()

,wall,lives,building,infrastructure,road,sidewalk,sky,green,transportation,publicservice,crime_counts
0,0.186966,0.096057,0.262003,0.209962,0.325591,0.305797,0.633910,0.254651,0.114748,0.015881,5
1,0.289723,0.262073,0.591425,0.099004,0.297520,0.331383,0.454224,0.104795,0.021102,0.003712,0
2,0.208656,0.102035,0.286452,0.085730,0.652819,0.565437,0.051069,0.490026,0.039030,0.001341,8
3,0.116361,0.046017,0.532984,0.045333,0.756146,0.680791,0.214335,0.129281,0.174739,0.008353,28
4,0.317734,0.089263,0.222944,0.403581,0.098033,0.036346,0.199131,0.581268,0.128060,0.005878,2


In [488]:
selectCrimeCI = pd.read_csv("./correlation_ready.csv",index_col=0)
selectCrimeCI = selectCrimeCI.drop(columns=['pointId'])
Vars=["wall","lives","building","infrastructure","road","sidewalk","sky","green","transportation","publicservice"]
selectCrimeCI[Vars] = selectCrimeCI[Vars].apply(lambda x: (x-x.min())/(x.max()-x.min()))

In [489]:
X = selectCrimeCI[Vars].to_numpy()
X = sm.add_constant(X)
y = selectCrimeCI['crime_counts'].to_numpy()

In [490]:
model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.093
Model:                            OLS   Adj. R-squared:                  0.092
Method:                 Least Squares   F-statistic:                     126.5
Date:                Sun, 19 Dec 2021   Prob (F-statistic):          1.68e-252
Time:                        12:46:07   Log-Likelihood:                -59965.
No. Observations:               12317   AIC:                         1.200e+05
Df Residuals:                   12306   BIC:                         1.200e+05
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         59.9745      8.608      6.967      0.0

In [539]:
regressionOutcome = pd.DataFrame(results.params)
VARS = pd.Series(["intercept"]).append(pd.Series(Vars)).reset_index(drop=True)
lowCI = ["intercept","-5.840","4.331",'-20.169','-9.604',"-11.021"," -8.502","-48.509","-75.801","-1.347","46.378"]
highCI = ["intercept",'6.276','16.489','5.769','1.569',"2.704","2.132","-31.044","-46.761","9.941","96.606"]
coefficient = regressionOutcome.iloc[:,0]

regressionOutcome["vars"] = VARS
regressionOutcome["lowCI"] = lowCI
regressionOutcome["highCI"] = highCI
regressionOutcome = regressionOutcome[1:]
regressionOutcome["coefficient"] = coefficient
# regressionOutcome = regressionOutcome.rename(columns={"0": "coefficient"}, errors="raise")
regressionOutcome = regressionOutcome.iloc[:,1:5]
regressionOutcome = pd.DataFrame(regressionOutcome)
regressionOutcome

,vars,lowCI,highCI,coefficient
1,wall,-5.840,6.276,0.217647
2,lives,4.331,16.489,10.409833
3,building,-20.169,5.769,-7.199650
4,infrastructure,-9.604,1.569,-4.017654
5,road,-11.021,2.704,-4.158596
6,sidewalk,-8.502,2.132,-3.185238
7,sky,-48.509,-31.044,-39.776251
8,green,-75.801,-46.761,-61.281028
9,transportation,-1.347,9.941,4.297176
10,publicservice,46.378,96.606,71.491967


In [638]:
coefficient = alt.Chart(regressionOutcome).mark_point(fill="#72B7B3"
).encode(
    x=alt.X('coefficient:Q',axis=alt.Axis(title="Coefficient",ticks=False, domain=False)),
    y=alt.Y('vars:N',axis=alt.Axis(title="Variables",ticks=False, domain=False)),
    color = alt.value('#72B7B3'),
    size=alt.value(40)
)

CI = coefficient.mark_line().encode(
    y='vars:N',
    x="lowCI:Q",
    x2="highCI:Q",
    color = alt.value('#F55218'),
    size=alt.value(2)
)

CorrelationPlot = (CI + coefficient)

# Scatter Plot

In [649]:
import altair as alt
from vega_datasets import data
alt.data_transformers.enable('json')
Vars=["wall","lives","building","infrastructure","road","sidewalk","sky","green","transportation","publicservice"]
textbox = ["0.217647","10.409833","-7.199650","-4.017654","-4.158596","-3.185238","-39.776251","-61.281028","4.297176","71.491967"]

charts=[]
for i,s in enumerate(Vars):
    selection = selectCrime2[(selectCrime2["variable"]==s)]
    selection = selection[(selection["crime_counts"]<1200)]
    chart_single=alt.Chart(selection).mark_point().encode(
    x=alt.X('value:Q',axis=alt.Axis(title='{}'.format(s),ticks=False, domain=False)),
    y=alt.Y('crime_counts:Q',axis=alt.Axis(title='Crime Counts',ticks=False, domain=False)),
    opacity=alt.value(0.4),
    color = alt.value('#72B7B3')
    )
    text = alt.Chart({'values':[{'x': 0.1, 'y': 750}]}).mark_text(text=textbox[i]).encode(color = alt.value('#F55218'),x="x:Q", y="y:Q")
    chart=chart_single + chart_single.transform_regression('value','crime_counts').mark_line().encode(
        color = alt.value('#F55218'),
        opacity=alt.value(1),
    strokeDash=alt.value([5, 5])) + text
    charts.append(chart)


In [666]:
All = (charts[0]|charts[1]|charts[2]|charts[3]|charts[4])&(charts[5]|charts[6]|charts[7]|charts[8]|charts[9])&CorrelationPlot
# + chart.transform_regression('value:Q','crime_counts:Q').mark_line()
All

alt.VConcatChart(...)

In [664]:
All.to_dict()

{'config': {'view': {'continuousWidth': 400, 'continuousHeight': 300}},
 'vconcat': [{'hconcat': [{'layer': [{'mark': 'point',
       'encoding': {'color': {'value': '#72B7B3'},
        'opacity': {'value': 0.4},
        'x': {'type': 'quantitative',
         'axis': {'domain': False, 'ticks': False, 'title': 'wall'},
         'field': 'value'},
        'y': {'type': 'quantitative',
         'axis': {'domain': False, 'ticks': False, 'title': 'Crime Counts'},
         'field': 'crime_counts'}}},
      {'mark': 'line',
       'encoding': {'color': {'value': '#F55218'},
        'opacity': {'value': 1},
        'strokeDash': {'value': [5, 5]},
        'x': {'type': 'quantitative',
         'axis': {'domain': False, 'ticks': False, 'title': 'wall'},
         'field': 'value'},
        'y': {'type': 'quantitative',
         'axis': {'domain': False, 'ticks': False, 'title': 'Crime Counts'},
         'field': 'crime_counts'}},
       'transform': [{'on': 'value', 'regression': 'crime_counts'}